# 05 - D3.js Custom Visualization

## Learning Objectives
- Learn to configure D3.js library using jsConfig
- Master D3.js data binding and transformation concepts
- Create custom SVG visualizations with D3.js
- Understand animations, transitions, and interactive features

## Prerequisites
- Completed Example 04 (ECharts Advanced Charts)
- Understanding of SVG and DOM manipulation
- Familiarity with D3.js selection and data binding

## Difficulty: ⭐⭐⭐⭐☆

## Core Concept

This example demonstrates creating custom visualizations using D3.js:

```
Kotlin complex data → jsExport() → JavaScript import → D3.js data binding → Custom SVG rendering
```


In [6]:
// 🔧 Local debug version
USE {
    repositories {
        mavenLocal()
        mavenCentral()
    }
    dependencies {
        implementation("dev.yidafu.jupyter:jupyter-js:0.8.0")
    }
}

## Step 1: Generate Complex Data Structures

Create data suitable for D3.js custom visualizations.


In [7]:
// 1. Population data for bar chart
val populationData = listOf(
    mapOf("country" to "China", "population" to 1439323776),
    mapOf("country" to "India", "population" to 1380004385),
    mapOf("country" to "USA", "population" to 331002651),
    mapOf("country" to "Indonesia", "population" to 273523615),
    mapOf("country" to "Pakistan", "population" to 220892340),
    mapOf("country" to "Brazil", "population" to 212559417)
)

// 2. Network graph data
val networkNodes = listOf(
    mapOf("id" to "A", "group" to 1, "value" to 30),
    mapOf("id" to "B", "group" to 1, "value" to 25),
    mapOf("id" to "C", "group" to 2, "value" to 20),
    mapOf("id" to "D", "group" to 2, "value" to 15),
    mapOf("id" to "E", "group" to 3, "value" to 18),
    mapOf("id" to "F", "group" to 3, "value" to 22)
)

val networkLinks = listOf(
    mapOf("source" to "A", "target" to "B", "value" to 5),
    mapOf("source" to "B", "target" to "C", "value" to 3),
    mapOf("source" to "C", "target" to "D", "value" to 4),
    mapOf("source" to "D", "target" to "E", "value" to 2),
    mapOf("source" to "E", "target" to "F", "value" to 3),
    mapOf("source" to "A", "target" to "E", "value" to 6)
)

// 3. Hierarchical data for tree visualization
val treeData = mapOf(
    "name" to "Root",
    "children" to listOf(
        mapOf(
            "name" to "Branch A",
            "children" to listOf(
                mapOf("name" to "Leaf A1", "value" to 10),
                mapOf("name" to "Leaf A2", "value" to 15)
            )
        ),
        mapOf(
            "name" to "Branch B",
            "children" to listOf(
                mapOf("name" to "Leaf B1", "value" to 20),
                mapOf("name" to "Leaf B2", "value" to 25)
            )
        )
    )
)

// 4. Scatter plot data
val scatterData = listOf(
    mapOf("x" to 10, "y" to 20, "category" to "A"),
    mapOf("x" to 15, "y" to 30, "category" to "A"),
    mapOf("x" to 20, "y" to 25, "category" to "B"),
    mapOf("x" to 25, "y" to 40, "category" to "B"),
    mapOf("x" to 30, "y" to 35, "category" to "C"),
    mapOf("x" to 35, "y" to 50, "category" to "C")
)

println("✅ D3.js visualization data created successfully")
println("Ready for: Bar chart, Network graph, Tree diagram, Scatter plot")


✅ D3.js visualization data created successfully
Ready for: Bar chart, Network graph, Tree diagram, Scatter plot


## Step 2: Create Custom Bar Chart with D3.js

Build a bar chart with custom styling and animations.


In [8]:
%js

import { populationData } from '@jupyter';
import * as d3 from 'd3';

const container = getContainer();
container.innerHTML = '<svg id="bar-chart"></svg>';

const svg = d3.select("#bar-chart");
const margin = { top: 20, right: 30, bottom: 40, left: 60 };
const width = 800 - margin.left - margin.right;
const height = 400 - margin.top - margin.bottom;

svg.attr("width", width + margin.left + margin.right)
   .attr("height", height + margin.top + margin.bottom);

const g = svg.append("g")
    .attr("transform", `translate(${margin.left},${margin.top})`);

// Create scales
const xScale = d3.scaleBand()
    .domain(populationData.map(d => d.country))
    .range([0, width])
    .padding(0.1);

const yScale = d3.scaleLinear()
    .domain([0, d3.max(populationData, d => d.population)])
    .range([height, 0]);

// Create bars with animation
const bars = g.selectAll(".bar")
    .data(populationData)
    .enter().append("rect")
    .attr("class", "bar")
    .attr("x", d => xScale(d.country))
    .attr("width", xScale.bandwidth())
    .attr("y", height)
    .attr("height", 0)
    .attr("fill", "#3498db");

// Animate bars
bars.transition()
    .duration(1000)
    .attr("y", d => yScale(d.population))
    .attr("height", d => height - yScale(d.population));

// Add axes
g.append("g")
    .attr("transform", `translate(0,${height})`)
    .call(d3.axisBottom(xScale))
    .selectAll("text")
    .style("text-anchor", "end")
    .attr("dx", "-.8em")
    .attr("dy", ".15em")
    .attr("transform", "rotate(-45)");

g.append("g")
    .call(d3.axisLeft(yScale).tickFormat(d => (d / 1000000).toFixed(1) + "M"));

// Add title
svg.append("text")
    .attr("x", width / 2 + margin.left)
    .attr("y", 15)
    .attr("text-anchor", "middle")
    .style("font-size", "18px")
    .style("font-weight", "bold")
    .text("Population by Country");

console.log('Bar chart created with D3.js:', populationData);


In [9]:
%js

import { scatterData } from '@jupyter';
import * as d3 from 'd3';

const container = getContainer();
container.innerHTML = '<svg id="scatter-plot"></svg>';

const svg = d3.select("#scatter-plot");
const margin = { top: 20, right: 30, bottom: 40, left: 50 };
const width = 600 - margin.left - margin.right;
const height = 400 - margin.top - margin.bottom;

svg.attr("width", width + margin.left + margin.right)
   .attr("height", height + margin.top + margin.bottom);

const g = svg.append("g")
    .attr("transform", `translate(${margin.left},${margin.top})`);

// Create scales
const xScale = d3.scaleLinear()
    .domain([0, d3.max(scatterData, d => d.x) * 1.1])
    .range([0, width]);

const yScale = d3.scaleLinear()
    .domain([0, d3.max(scatterData, d => d.y) * 1.1])
    .range([height, 0]);

// Color scale
const colorScale = d3.scaleOrdinal()
    .domain(["A", "B", "C"])
    .range(["#e74c3c", "#3498db", "#2ecc71"]);

// Create circles
const circles = g.selectAll("circle")
    .data(scatterData)
    .enter().append("circle")
    .attr("cx", d => xScale(d.x))
    .attr("cy", d => yScale(d.y))
    .attr("r", 0)
    .attr("fill", d => colorScale(d.category))
    .attr("opacity", 0.7)
    .attr("stroke", "#fff")
    .attr("stroke-width", 2);

// Animate circles
circles.transition()
    .duration(1000)
    .attr("r", 8);

// Add axes
g.append("g")
    .attr("transform", `translate(0,${height})`)
    .call(d3.axisBottom(xScale));

g.append("g")
    .call(d3.axisLeft(yScale));

// Add labels
svg.append("text")
    .attr("x", width / 2 + margin.left)
    .attr("y", 15)
    .attr("text-anchor", "middle")
    .style("font-size", "18px")
    .style("font-weight", "bold")
    .text("Scatter Plot");

svg.append("text")
    .attr("x", width / 2 + margin.left)
    .attr("y", height + margin.top + margin.bottom - 5)
    .attr("text-anchor", "middle")
    .text("X Value");

svg.append("text")
    .attr("transform", "rotate(-90)")
    .attr("y", 15)
    .attr("x", -height / 2 - margin.top)
    .attr("text-anchor", "middle")
    .text("Y Value");

console.log('Scatter plot created with D3.js:', scatterData);


## Step 4: Create Tree Diagram

Visualize hierarchical data as a tree structure.


In [10]:
%js

import { treeData } from '@jupyter';
import * as d3 from 'd3';

const container = getContainer();
container.innerHTML = '<svg id="tree-diagram"></svg>';

const svg = d3.select("#tree-diagram");
const width = 600;
const height = 400;

svg.attr("width", width)
   .attr("height", height);

const g = svg.append("g")
    .attr("transform", "translate(40,20)");

// Create tree layout
const tree = d3.tree()
    .size([height - 40, width - 100]);

const root = d3.hierarchy(treeData);
tree(root);

// Create links
const links = g.selectAll(".link")
    .data(root.links())
    .enter().append("path")
    .attr("class", "link")
    .attr("d", d3.linkHorizontal()
        .x(d => d.y)
        .y(d => d.x))
    .attr("fill", "none")
    .attr("stroke", "#ccc")
    .attr("stroke-width", 2);

// Create nodes
const nodes = g.selectAll(".node")
    .data(root.descendants())
    .enter().append("g")
    .attr("class", "node")
    .attr("transform", d => `translate(${d.y},${d.x})`);

nodes.append("circle")
    .attr("r", 8)
    .attr("fill", d => d.children ? "#555" : "#999");

nodes.append("text")
    .attr("dy", ".35em")
    .attr("x", d => d.children ? -13 : 13)
    .style("text-anchor", d => d.children ? "end" : "start")
    .text(d => d.data.name);

svg.append("text")
    .attr("x", width / 2)
    .attr("y", 15)
    .attr("text-anchor", "middle")
    .style("font-size", "18px")
    .style("font-weight", "bold")
    .text("Tree Diagram");

console.log('Tree diagram created with D3.js:', treeData);


## Summary

This example demonstrated:

1. ✅ **D3.js configuration** - Setting up D3.js library with jsConfig
2. ✅ **Data binding** - Using D3.js selection and data binding patterns
3. ✅ **Custom visualizations** - Building bar charts, scatter plots, and tree diagrams
4. ✅ **Animations** - Adding transitions and interactive features

## Extension Exercises

- Add interactive tooltips to visualizations
- Implement zoom and pan features
- Create force-directed graph layouts
- Add data filtering and dynamic updates
- Create custom color schemes and themes
